In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools
import os
import pandas as pd
import tensorflow as tf
import urllib

In [6]:
BOSTON_TRAIN       = "boston_train.csv"
BOSTON_TRAIN_URL   = "http://download.tensorflow.org/data/boston_train.csv"
BOSTON_TEST        = "boston_test.csv"
BOSTON_TEST_URL    = "http://download.tensorflow.org/data/boston_test.csv"
BOSTON_PREDICT     = "boston_predict.csv"
BOSTON_PREDICT_URL = "http://download.tensorflow.org/data/boston_predict.csv"

COLUMNS  = ["crim", "zn", "indus", "nox", "rm", "age", "dis", "tax", "ptratio", "medv"]
FEATURES = ["crim", "zn", "indus", "nox", "rm", "age", "dis", "tax", "ptratio"]
LABEL    = "medv"

def download_data(url_file, path):
    if os.path.exists(path):
        return False
    
    raw = urllib.request.urlopen(url_file).read()
    with open(path, "wb") as f: f.write(raw)
    return True

def input_fn(data_set):
    feature_cols = {k: tf.constant(data_set[k].values) for k in FEATURES}
    labels       = tf.constant(data_set[LABEL].values)
    return feature_cols, labels
            
def run():
    download_data(BOSTON_TRAIN_URL,   BOSTON_TRAIN)
    download_data(BOSTON_TEST_URL,    BOSTON_TEST)
    download_data(BOSTON_PREDICT_URL, BOSTON_PREDICT)
    
    tf.logging.set_verbosity(tf.logging.INFO)
    train_set = pd.read_csv(
        "boston_train.csv", skipinitialspace=True, skiprows=1, names=COLUMNS)
    test_set = pd.read_csv(
        "boston_test.csv", skipinitialspace=True, skiprows=1, names=COLUMNS)
    predict_set = pd.read_csv(
        "boston_predict.csv", skipinitialspace=True, skiprows=1, names=COLUMNS)
    
    feature_cols = [tf.contrib.layers.real_valued_column(k) for k in FEATURES]
    regressor    = tf.contrib.learn.DNNRegressor(
        feature_columns=feature_cols, hidden_units=[10, 10], model_dir="/tmp/boston")
    
    regressor.fit(input_fn=lambda: input_fn(train_set), steps=5000)
    
    evaluate   = regressor.evaluate(input_fn=lambda: input_fn(test_set), steps=1)
    loss_score = evaluate["loss"]
    print("Loss: {0:f}".format(loss_score))
    
    y = regressor.predict(input_fn=lambda: input_fn(predict_set))
    predictions = list(itertools.islice(y, 6))
    print("Predictions: {}".format(str(predictions)))

if __name__ == "__main__":
    run()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe7938aef98>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/boston'}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create Checkp

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-07-19-11:45:26
INFO:tensorflow:Restoring parameters from /tmp/boston/model.ckpt-40000
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-07-19-11:45:27
INFO:tensorflow:Saving dict for global step 40000: global_step = 40000, loss = 9.30219
Loss: 9.302193
Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.
INFO:tensorflow:Restoring parameters from /tmp/boston/model.ckpt-40000
Predictions: [34.117268, 19.918444, 24.626377, 35.929752, 17.448057, 17.999817]
